## Templating

In [ ]:
from liom_toolkit.registration import build_template_for_resolution
from tqdm.auto import tqdm

In [ ]:
# Set the brains to use for the template
brain_files = [
    "/data/LSFM/S18/647nm.zarr",
    "/data/LSFM/S19/647nm.zarr",
    "/data/LSFM/S21/647nm.zarr",
    "/data/LSFM/S23/647nm.zarr"
]

# Set the name of the brains (use for saving intermediate results).
# The order of the names should match the order of the brains, mirrored version will be added automatically if needed.
brain_names = [
    "S18",
    "S19",
    "S21",
    "S23"
]

In [ ]:
# Set the resolution levels (zarr index) and the matching atlas resolution
resolution_levels = [2, 3]
atlas_resolutions = [25, 50]
iterations = 15
# This decides if the first brain used in the templating process is the template or the first brain in the list
init_with_template = False
# This decides if the new template is registered to the allen atlas after the templating process
register_to_template = False
# This decides if the mirrored brains are used in the templating process
flipped_brains = False

In [ ]:
# Build the template for the different resolutions
for (resolution_level, atlas_resolution) in (
        pbar := tqdm(zip(resolution_levels, atlas_resolutions), desc="Building templates", leave=True,
                     total=len(resolution_levels), unit="template", position=0)):
    pbar.set_description(f"Building template at {atlas_resolution} microns")
    output_file = f"/data/LSFM/template/p11/p11_lightsheet_{atlas_resolution}microns.nii"
    build_template_for_resolution(output_file=output_file, zarr_files=brain_files,
                                  brain_names=brain_names,
                                  resolution_level=resolution_level,
                                  template_resolution=atlas_resolution, iterations=iterations,
                                  init_with_template=init_with_template,
                                  register_to_template=register_to_template,
                                  flipped_brains=flipped_brains)